# Getting Started with pandas

`pandas` will be a major tool of interest throughout much of the rest of the book. It contains data structures and data manipulation tools designed to make data cleaning and analysis fast and easy in Python. pandas is often used in tandem with numerical computing tools like NumPy and SciPy, analytical libraries like statsmodels and scikit-learn, and data visualization libraries like matplotlib. pandas adopts significant parts of NumPy’s idiomatic style of array-based computing, especially array-based functions and a preference for data processing without for loops.

While pandas adopts many coding idioms from NumPy, **the biggest difference is that pandas is designed for working with tabular or heterogeneous data**. NumPy, by contrast, is best suited for working with homogeneous numerical array data.

Since becoming an open source project in 2010, pandas has matured into a quite large library that’s applicable in a broad set of real-world use cases. The developer community has grown to over 800 distinct contributors, who’ve been helping build the project as they’ve used it to solve their day-to-day data problems.

Throughout the rest of the book, I use the following import convention for pandas:

In [ ]:
import pandas as pd

In [ ]:
from pandas import Series, DataFrame

In [ ]:
import numpy as np

## 1. Introduction to pandas Data Structures

To get started with pandas, you will need to get comfortable with its two workhorse data structures: Series and DataFrame. While they are not a universal solution for every problem, they provide a solid, easy-to-use basis for most applications.

### Series
A Series is a one-dimensional array-like object containing a sequence of values (of similar types to NumPy types) and an associated array of data labels, called its index. The simplest Series is formed from only an array of data:

In [ ]:
obj = pd.Series([4, 7, -5, 3])

In [ ]:
obj

The string representation of a Series displayed interactively shows the index on the left and the values on the right. Since we did not specify an index for the data, a default one consisting of the integers 0 through N - 1 (where N is the length of the data) is created. You can get the array representation and index object of the Series via its values and index attributes, respectively:

In [ ]:
obj.values

In [ ]:
obj.index # like range(4)

Often it will be desirable to create a Series with an index identifying each data point with a label:

In [ ]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

In [ ]:
obj2

In [ ]:
obj2.values

In [ ]:
obj2.index

Compared with NumPy arrays, you can use labels in the index when selecting single values or a set of values:

In [ ]:
obj2['a']

In [ ]:
obj2['b']

In [ ]:
obj2[['a', 'b', 'd', 'c']]

Here `['c', 'a', 'd']` is interpreted as a list of indices, even though it contains strings instead of integers.

Using NumPy functions or NumPy-like operations, such as filtering with a boolean array, scalar multiplication, or applying math functions, will preserve the index-value link:

In [ ]:
obj2[obj2 > 0]

In [ ]:
obj2 * 2

In [ ]:
np.exp(obj2)

Another way to think about a Series is as a fixed-length, ordered dict, as it is a mapping of index values to data values. It can be used in many contexts where you might use a dict:

In [ ]:
'b' in obj2

In [ ]:
'e' in obj2

In [ ]:
 'e' in obj2 or 'b' in obj2

Should you have data contained in a Python dict, you can create a Series from it by passing the dict:

In [ ]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

In [ ]:
obj3 = pd.Series(sdata)

In [ ]:
obj3

When you are only passing a dict, the index in the resulting Series will have the dict’s keys in sorted order. You can override this by passing the dict keys in the order you want them to appear in the resulting Series:

In [ ]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)
obj4

Here, three values found in sdata were placed in the appropriate locations, but since no value for 'California' was found, it appears as NaN (not a number), which is considered in pandas to mark missing or NA values. Since 'Utah' was not included in states, it is excluded from the resulting object.

I will use the terms “missing” or “NA” interchangeably to refer to missing data. The `isnull` and `notnull` functions in pandas should be used to detect missing data:

In [ ]:
pd.isnull(obj4)

In [ ]:
pd.notnull(obj4)

Series also have these as instance methods:

In [ ]:
obj.isnull()

A useful Series feature for many applications is that it automatically aligns by index label in arithmetic operations:

In [ ]:
obj3

In [ ]:
obj4

In [ ]:
obj3 + obj4

Data alignment features will be addressed in more detail later. If you have experience with databases, you can think about this as being similar to a join operation.

Both the Series object itself and its index have a `name` attribute, which integrates with other key areas of pandas functionality:

In [ ]:
obj4.name = 'population'

In [ ]:
obj4.index.name = 'state'

In [ ]:
obj4

A Series’s index can be altered in-place by assignment:

In [ ]:
obj

In [ ]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']

In [ ]:
obj

### DataFrame

A DataFrame represents a rectangular table of data and contains an ordered collection of columns, each of which can be a different value type (numeric, string, boolean, etc.). The DataFrame has both a row and column index; it can be thought of as a dict of Series all sharing the same index. Under the hood, the data is stored as one or more two-dimensional blocks rather than a list, dict, or some other collection of one-dimensional arrays. The exact details of DataFrame’s internals are outside the scope of this book.

> **Note:** *While a DataFrame is physically two-dimensional, you can use it to represent higher dimensional data in a tabular format using hierarchical indexing, an ingredient in some of the more advanced data-handling features in pandas.*

There are many ways to construct a DataFrame, though one of the most common is from a dict of equal-length lists or NumPy arrays:

In [ ]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

In [ ]:
frame

For large DataFrames, the `head` method selects only the first five rows:

In [ ]:
frame.head()

If you specify a sequence of columns, the DataFrame’s columns will be arranged in that order:

In [ ]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

If you pass a column that isn’t contained in the dict, it will appear with missing values in the result:

In [ ]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'], 
                      index=['one', 'two', 'three', 'four', 'five', 'six'])


In [ ]:
frame2

In [ ]:
frame2.columns

A column in a DataFrame can be retrieved as a Series either by dict-like notation or by attribute:

In [ ]:
frame2['state']

In [ ]:
frame2.year

>**Note:** *Attribute-like access (e.g., frame2.year) and tab completion of column names in IPython is provided as a convenience. frame2[column] works for any column name, but frame2.column only works when the column name is a valid Python variable name.*

Note that the returned Series have the same index as the DataFrame, and their name attribute has been appropriately set.

Rows can also be retrieved by position or name with the special loc attribute (much more on this later):

In [ ]:
frame2.loc['three']

Columns can be modified by assignment. For example, the empty 'debt' column could be assigned a scalar value or an array of values:

In [ ]:
frame2['debt'] = 16.5

In [ ]:
frame2

In [ ]:
frame2['debt'] = np.arange(6.)

In [ ]:
frame2

When you are assigning lists or arrays to a column, the value’s length must match the length of the DataFrame. If you assign a Series, its labels will be realigned exactly to the DataFrame’s index, inserting missing values in any holes:

In [ ]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])

In [ ]:
val

In [ ]:
frame2['debt'] = val

In [ ]:
frame2

Assigning a column that doesn’t exist will create a new column. The del keyword will delete columns as with a dict.

As an example of del, I first add a new column of boolean values where the state column equals 'Ohio':

In [ ]:
frame2['eastern'] = frame2.state == 'Ohio'

In [ ]:
frame2

>**Caution:** *New columns cannot be created with the frame2.eastern syntax.*

The `del` method can then be used to remove this column:

In [ ]:
del frame2['eastern']

In [ ]:
frame2

>**Caution:** *The column returned from indexing a DataFrame is a view on the underlying data, not a copy. Thus, any in-place modifications to the Series will be reflected in the DataFrame. The column can be explicitly copied with the Series’s copy method.*

Another common form of data is a nested dict of dicts:

In [ ]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9}, 'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

If the nested dict is passed to the DataFrame, pandas will interpret the outer dict keys as the columns and the inner keys as the row indices:

In [ ]:
frame3 = pd.DataFrame(pop)

In [ ]:
frame3

You can transpose the DataFrame (swap rows and columns) with similar syntax to a NumPy array:

In [ ]:
frame3.T

The keys in the inner dicts are combined and sorted to form the index in the result. This isn’t true if an explicit index is specified:

In [ ]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

Dicts of Series are treated in much the same way:

In [ ]:
pdata = {'Ohio': frame3['Ohio'][:-1], 'Nevada': frame3['Nevada'][:2]}

In [ ]:
pd.DataFrame(pdata)

For a complete list of things you can pass the DataFrame constructor, see the table.

If a DataFrame’s index and columns have their name attributes set, these will also be displayed:

In [ ]:
frame3.index.name = 'year'; frame3.columns.name = 'state'

In [ ]:
frame3

As with Series, the values attribute returns the data contained in the DataFrame as a two-dimensional ndarray:

In [ ]:
frame3.values

If the DataFrame’s columns are different dtypes, the dtype of the values array will be chosen to accommodate all of the columns:

In [ ]:
frame2.values

*Possible data inputs to DataFrame constructor*

| Type | Notes |
|:---  |:---   |
| 2D ndarray | A matrix of data, passing optional row and column labels |
| dict of arrays, lists, or tuples | Each sequence becomes a column in the DataFrame; all sequences must be the same length |
| NumPy structured/record array | Treated as the “dict of arrays” case |
| dict of Series | Each value becomes a column; indexes from each Series are unioned together to form the result’s row index if no explicit index is passed |
| dict of dicts | Each inner dict becomes a column; keys are unioned to form the row index as in the “dict of Series” case |
| List of dicts or Series | Each item becomes a row in the DataFrame; union of dict keys or Series indexes become the DataFrame’s column labels |
| List of lists or tuples | Treated as the “2D ndarray” case |
| Another DataFrame | The DataFrame’s indexes are used unless different ones are passed |
| NumPy MaskedArray | Like the “2D ndarray” case except masked values become NA/missing in the DataFrame result |

### Index Objects

pandas’s Index objects are responsible for holding the axis labels and other metadata (like the axis name or names). Any array or other sequence of labels you use when constructing a Series or DataFrame is internally converted to an Index:

In [ ]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])

In [ ]:
index = obj.index

In [ ]:
index

In [ ]:
index[1:]

Index objects are immutable and thus can’t be modified by the user:

In [ ]:
# index[1] = 'd' # TypeError

Immutability makes it safer to share Index objects among data structures:

In [ ]:
labels = pd.Index(np.arange(3))

In [ ]:
labels

In [ ]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)

In [ ]:
obj2

In [ ]:
obj2.index is labels

In [ ]:
obj2.index

>**Caution:** *Some users will not often take advantage of the capabilities provided by indexes, but because some operations will yield results containing indexed data, it’s important to understand how they work.*

In addition to being array-like, an Index also behaves like a fixed-size set:

In [ ]:
frame3

In [ ]:
frame3.columns

In [ ]:
'Ohio' in frame3.columns

In [ ]:
2003 in frame3.index

Unlike Python sets, a pandas Index can contain duplicate labels:

In [ ]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])

In [ ]:
dup_labels

Selections with duplicate labels will select all occurrences of that label.

Each Index has a number of methods and properties for set logic, which answer other common questions about the data it contains. Some useful ones are summarized in the following table:

| Method | Description |
|:---    |:---         |
| `append` | Concatenate with additional Index objects, producing a new Index |
| `difference` | Compute set difference as an Index |
| `intersection` | Compute set intersection |
| `union` | Compute set union |
| `isin` | Compute boolean array indicating whether each value is contained in the passed collection |
| `delete` | Compute new Index with element at index i deleted |
| `drop` | Compute new Index by deleting passed values |
| `insert` | Compute new Index by inserting element at index i |
| `is_monotonic` | Returns True if each element is greater than or equal to the previous element |
| `is_unique` | Returns True if the Index has no duplicate values |
| `unique` | Compute the array of unique values in the Index |